In [4]:
import os
import getpass
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.chains.question_answering import load_qa_chain
import keys

OPENAI_API_KEY = keys.OPENAI_API_KEY
PINECONE_API_KEY = keys.PINECONE_API_KEY
PINECONE_API_ENV = keys.PINECONE_API_ENV

#there are many options from the above that can be used. Langchain offeres many libraries that allow users to interact with OpenAIs LLMS given that they have an API key 
#for this code however, the lines from embeddings, chat_models, memory, and qa chain are used
# embeddings are used to convert the textual infroamtion in documents into vector data that the LLM can understand and query using the semantics in a users query to find relevant results
# ChatOpenAI gives us the ability to choose which LLM we can use. The LLM will look into the vector databse and contruct an answer. We will use our fine tuned instance once we find a way to make it accessible throught this code
# ChatMessage History allows use to build a history object where the LLM keeps track of user questions and AI responses to keep the conversation relevant
# load_qa_chain is the question an answer chain that allows us to run the a user quer with the vector database. usually this function returns a generic response. but the code below has an input documents similarity parameter to get relevant information



# always remove the API key before committing code to github. 


embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# temperature in the function below is 0. this is the randomness variable. a temperature value > 1 will be completely random. 
# the value below is at 0 to have no randomness. this coupled with our relevant document search is our method of mitigating hallucinations from the LLM

#llm = ChatOpenAI(model='gpt-3.5-turbo',temperature=0, openai_api_key=API_Key)
model_name = 'gpt-3.5-turbo'
llm = ChatOpenAI(model=model_name,temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type='stuff')

# initializing history to track the conversation

history = ChatMessageHistory()


# the value below only shows the messages from the entire conversation history
history.messages

query = ''


In [5]:
# our database value is vectordb. we are using the file db2 which has our pdfs and CSVs. we had db and db (csv) as test cases aginst information in pdfs and CSVs. 
# we did not know how a vector databse would behave if we gave put differently structued data in one place. but its working out well so far. 

vectordb = Chroma(persist_directory='db2', embedding_function=embeddings)

In [6]:
# assigning a retriever function to a value. this allows the program to retrieve documents for our similartiy search which matches the semantics of our query to info in the databse
retriever = vectordb.as_retriever()

In [7]:
# the below is our current chat backend. in later sprints we will have user interface. but the below is fine for our testing purposes
# if the user types exit, the loop ends

while True:
    query = str(input("Human: "))
    if query == 'exit':
        break
    # adding a human message to history
    history.add_user_message(query)
    print(f'Human: {query}')
    print()
    validity = len(retriever.get_relevant_documents(query))
    # the below if statment will probably never hit. semantics so far have never caused no documents to show up. 
    if validity == 0:
        response = chat(history.messages)
        print("question cannot be answered by database")
        print(response.content)
        history.add_ai_message(response.content)
        history.messages
    # in the below else statement, we have a similarity value that will take semantics from the databse that match our query
    # the chain.run statement from qa chain will generate a response using both similarity and quer to thier respective values in the function
    else:
        similarity = vectordb.similarity_search(query, k=10)
        # uncomment the below to enable debug mode
        
        print('--------------------------')
        print('debug mode start')
        print()
        print(f'Simlar dosuments are: {similarity}')
        print()
        print(f'number of documents: {validity}')
        print()
        print('debug mode end')
        print('---------------------------')
        print()
        
        response = chain.run(input_documents=similarity, question=query)
        print(f'Chat CSEC: {response}')
        print()
    # adding the response as the AI message to add to the chat history
        history.add_ai_message(response)
        history.messages

Human: hey

--------------------------
debug mode start

Simlar dosuments are: [Document(page_content='}\n}\n}\n}\nreturn\n0;\n}', metadata={}), Document(page_content='1NA NA NA NA NA NA\n1NA NA NA NA NA NA\n1NA NA NA NA NA NA', metadata={}), Document(page_content='obtained byfollowing\n17', metadata={}), Document(page_content='instructions. ########', metadata={}), Document(page_content='instructions. ########', metadata={}), Document(page_content='instructions. ########', metadata={}), Document(page_content='instructions. ########', metadata={}), Document(page_content='instructions. ########', metadata={}), Document(page_content='16NA NA NA NA NA NA\n16NA NA NA NA NA NA\n16NA NA NA NA NA NA', metadata={}), Document(page_content='19NA NA NA NA NA NA\n19NA NA NA NA NA NA\n19NA NA NA NA NA NA\n19NA NA NA NA NA NA\n19NA NA NA NA NA NA\n19NA NA NA NA NA NA\n19NA NA NA NA NA NA\n19NA NA NA NA NA NA\n19NA NA NA NA NA NA\n19NA NA NA NA NA NA\n19NA NA NA NA NA NA\n19NA NA NA NA NA NA\n19NA NA